Case Study 3 [Loss Prediction Model]:

•	The aim of this exercise is to identify which customers have a higher potential for loss.

•	For this exercise, select only those records for which LossEstimate variable is greater than 0. 

•	Cross-check that all these accounts have touched 6 or more cycle delinquency levels during their tenure. Losses are expected only from accounts which have very high delinquencies.

•	Now create the target variable as LossEstimate as a percentage of Original UPB. This is a ratio variable.

•	Use the same set of independent variables considered for the Originations Model exercise.

•	Using the independent variables, build a regression model of your choice to help predict what percentage of original principal might flow into losses. Use a 60:40 split for Train and Test.

•	Report the adjusted R-square, RMSE and MAPE values for the model.

•	Also, provide a graphical comparison of actual vs. predicted on the test data to indicate the quality of the loss estimation model. You may use Credit Score deciles to report actual vs. expected loss for each Credit Score bin to display this graph.


In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics


In [2]:
df1=pd.read_csv('./Originations_Data.csv')
df1.head()


C:\Users\vinee\AppData\Local\Temp\ipykernel_6984\1486993399.py:1: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(r"C:\Users\vinee\OneDrive\Desktop\Case study\Originations_Data.csv")


,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area,Mortgage Insurance Percentage,Number of Units,Occupancy Status,Original Combined Loan-to-Value,Original Debt-to-Income Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only Indicator
0,654,201803,N,204802,NaN,0,1,P,77,35,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
2,757,201803,Y,204802,NaN,25,1,P,97,10,...,360,1,Other sellers,Other servicers,NaN,NaN,H,NaN,2,N
3,807,201803,N,203302,NaN,0,1,P,26,38,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,812,201803,N,203302,19340.0,0,1,P,14,14,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N


In [3]:
df2=pd.read_csv('./Behaviour_Data.csv')
df2.head()

,Loan Sequence Number,Delinquency_Bucket 201802,Delinquency_Bucket 201803,Delinquency_Bucket 201804,Delinquency_Bucket 201805,Delinquency_Bucket 201806,Delinquency_Bucket 201807,Delinquency_Bucket 201808,Delinquency_Bucket 201809,Delinquency_Bucket 201810,...,Delinquency_Bucket 202107,Delinquency_Bucket 202108,Delinquency_Bucket 202109,Delinquency_Bucket 202110,Delinquency_Bucket 202111,Delinquency_Bucket 202112,Delinquency_Bucket 202201,Delinquency_Bucket 202202,Delinquency_Bucket 202203,LossEstimate
0,F18Q10000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,F18Q10000052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,F18Q10000084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F18Q10000117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,F18Q10000140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
temp=pd.merge(df1,df2,how='inner',on='Loan Sequence Number')
df_merged=temp[temp['LossEstimate']>0]
df_merged.shape

(3402, 82)

In [5]:
delinquency_cols=df2.columns[1:51]
df_merged[delinquency_cols]=df_merged[delinquency_cols].fillna(0)
for index, row in df_merged.iterrows():
    delinquency_list=[row[col] for col in delinquency_cols]
    df_merged['Maximum_Delinquency']=max(delinquency_list)

df_merged=df_merged[df_merged['Maximum_Delinquency']>=6]

C:\Users\vinee\AppData\Local\Temp\ipykernel_6984\2131742546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[delinquency_cols]=df_merged[delinquency_cols].fillna(0)
C:\Users\vinee\AppData\Local\Temp\ipykernel_6984\2131742546.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['Maximum_Delinquency']=max(delinquency_list)


In [6]:
df_merged['Loss_to_UPB']=df_merged['LossEstimate']/df_merged['Original UPB']


THE INDEPENDENT VARIABLES ARE:
'Credit Score',

'First Time Homebuyer Flag',

'Metropolitan Statistical Area (MSA) Or Metropolitan Division',

'Mortgage Insurance Percentage (MI %)',

'Number of Units',

'Occupancy Status',

'Original Combined Loan-to-Value (CLTV)',

'Original Debt-to-Income (DTI) Ratio',

'Original UPB',

'Original Loan-to-Value (LTV)',

'Original Interest Rate',

'Channel',

'Prepayment Penalty Mortgage (PPM) Flag',

'Amortization Type (Formerly Product Type)',

'Property Type',

'Loan Purpose',

'Original Loan Term',

'Number of Borrowers',

'Super Conforming Flag',

'Program Indicator',

'HARP Indicator']

In [7]:
final_data=df_merged[['Credit Score',
'First Time Homebuyer Flag',
'Metropolitan Statistical Area',
'Mortgage Insurance Percentage',
'Number of Units',
'Occupancy Status',
'Original Combined Loan-to-Value',
'Original Debt-to-Income Ratio',
'Original UPB',
'Original Loan-to-Value',
'Original Interest Rate',
'Channel',
'Prepayment Penalty Mortgage Flag',
'Amortization Type',
'Property Type',
'Loan Purpose',
'Original Loan Term',
'Number of Borrowers',
'Super Conforming Flag',
'Program Indicator',
'HARP Indicator','Loss_to_UPB']]

In [8]:
numeric_cols=[col for col in final_data.columns if final_data[col].dtype!='object']
categorical_cols=[col for col in final_data.columns if final_data[col].dtype=='object']

One hot encoding for categorical columns

In [9]:
final_data=pd.get_dummies(final_data,columns=categorical_cols)

In [10]:
final_data['Metropolitan Statistical Area']=final_data['Metropolitan Statistical Area'].fillna(0)

In [11]:
final_data.isnull().sum()

Credit Score                          0
Metropolitan Statistical Area         0
Mortgage Insurance Percentage         0
Number of Units                       0
Original Combined Loan-to-Value       0
Original Debt-to-Income Ratio         0
Original UPB                          0
Original Loan-to-Value                0
Original Interest Rate                0
Original Loan Term                    0
Number of Borrowers                   0
Loss_to_UPB                           0
First Time Homebuyer Flag_N           0
First Time Homebuyer Flag_Y           0
Occupancy Status_I                    0
Occupancy Status_P                    0
Occupancy Status_S                    0
Channel_B                             0
Channel_C                             0
Channel_R                             0
Prepayment Penalty Mortgage Flag_N    0
Amortization Type_FRM                 0
Property Type_CO                      0
Property Type_CP                      0
Property Type_MH                      0


In [12]:
X=final_data.drop(columns='Loss_to_UPB')
Y=final_data['Loss_to_UPB']

In [13]:
x_train, x_test, y_train,y_test=train_test_split(X,Y,test_size=0.4)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2041, 34)
(2041,)
(1361, 34)
(1361,)


In [14]:
model=LinearRegression()

In [15]:
model.fit(x_train,y_train)

LinearRegression()

In [16]:
prediction=model.predict(x_test)
prediction

array([0.73275892, 0.73342527, 0.73771694, ..., 0.72120418, 0.72596245,
       0.73483659])

Report the adjusted R-square, RMSE and MAPE values for the model.

In [17]:
R2=metrics.r2_score(y_test,prediction)
n=y_test.shape[0]
p=len(X.columns)
adjusted_R2=1-((1-R2)*(n-1)/(n-p-1))    #1- ((1-R2) * (n-1)/(n-p-1))
RMSE=metrics.mean_squared_error(y_test,prediction,squared=False)
MAPE=metrics.mean_absolute_percentage_error(y_test,prediction)

print(f'Adjusted R-Square for the model is:{adjusted_R2}')
print(f'RMSE for teh model is {RMSE}')
print(f'MAPE for the model is {MAPE}')

Adjusted R-Square for the model is:0.03716068853626309
RMSE for teh model is 0.040681253773892326
MAPE for the model is 0.06370599956583022
